In [81]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import random
import ml_metrics as metrics
import xgboost as xgb
import matplotlib.pyplot as plt
%matplotlib inline

In [119]:
#train = pd.read_csv("50k_with_extracted_features.csv")
train = pd.read_csv("50k_train.csv")
test = pd.read_csv("50k_test.csv")
#test = pd.read_csv("50k_test_with_extracted_features.csv")

In [120]:
train.shape

(40000, 27)

In [121]:
test.shape

(100000, 27)

In [122]:
train = train.drop(train.columns[0], axis=1)
test = test.drop(test.columns[0], axis=1)

In [123]:
train = train.drop("srch_ci", axis = 1)
train = train.drop("srch_co", axis = 1)
train = train.drop("date_time", axis = 1)
test = test.drop("srch_ci", axis = 1)
test = test.drop("srch_co", axis = 1)
test = test.drop("date_time", axis = 1)

### Base Model

In [124]:
most_common_clusters = list(train.hotel_cluster.value_counts().head().index)

In [125]:
most_common_clusters

[91, 48, 42, 28, 59]

In [126]:
predictions = [most_common_clusters for i in range(test.shape[0])]
target = [[l] for l in test["hotel_cluster"]]
print(metrics.mapk(target, predictions, k=5))

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x112576128>>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/xgboost-0.6-py3.6.egg/xgboost/core.py", line 337, in __del__
    _check_call(_LIB.XGDMatrixFree(self.handle))
AttributeError: 'DMatrix' object has no attribute 'handle'


0.0721046666667


### XGBoost

In [127]:
train_xgb = train.drop("hotel_cluster", axis = 1)

In [128]:
test_xgb = test.drop("hotel_cluster", axis = 1)

In [101]:
xgb_clf = XGBClassifier()
xgb_clf.fit(train_xgb, train['hotel_cluster'].values)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [129]:
prediction = xgb_clf.predict_proba(test_xgb)

ValueError: feature_names mismatch: ['Unnamed: 0.1', 'site_name', 'posa_continent', 'user_location_country', 'user_location_region', 'user_location_city', 'orig_destination_distance', 'user_id', 'is_mobile', 'is_package', 'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id', 'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market', 'year', 'month', 'hotel_nights', 'srch_yr', 'srch_month', 'num_in_advance'] ['site_name', 'posa_continent', 'user_location_country', 'user_location_region', 'user_location_city', 'orig_destination_distance', 'user_id', 'is_mobile', 'is_package', 'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id', 'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market', 'year', 'month']
expected num_in_advance, srch_month, hotel_nights, srch_yr, Unnamed: 0.1 in input data

In [110]:
xgb_preds = []
for i in range(len(prediction)):
    xgb_preds.append(prediction[i].argsort()[-5:][::-1])

In [111]:
target = [[l] for l in test["hotel_cluster"]]
print(metrics.mapk(target, xgb_preds, k=5))

0.207815


In [112]:
importances = xgb_clf.feature_importances_
indices = np.argsort(importances)[::-1]
for i in indices:
    print(str(train_xgb.columns[i]) + ': ' + str(importances[i]))

hotel_market: 0.188394
srch_destination_id: 0.143741
hotel_country: 0.0832969
hotel_continent: 0.0739031
orig_destination_distance: 0.0737735
Unnamed: 0.1: 0.0539494
num_in_advance: 0.05152
user_id: 0.0510017
user_location_city: 0.047228
user_location_region: 0.0420938
hotel_nights: 0.0326839
month: 0.0225289
user_location_country: 0.0217353
srch_destination_type_id: 0.0202776
srch_adults_cnt: 0.0172327
srch_month: 0.0172003
site_name: 0.0129407
channel: 0.0121633
srch_children_cnt: 0.0102846
is_package: 0.00730447
posa_continent: 0.00677
srch_rm_cnt: 0.00518277
is_mobile: 0.00320684
srch_yr: 0.000858397
cnt: 0.000728828
is_booking: 0.0
year: 0.0


### Random Forest

In [113]:
train_rf = train.fillna(0)
test_rf = test.fillna(0)

In [114]:
rf_clf = RandomForestClassifier(n_estimators = 100)
rf_clf.fit(train_rf.drop("hotel_cluster", axis = 1), train_rf['hotel_cluster'].values)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [115]:
prediction = rf_clf.predict_proba(test_rf.drop("hotel_cluster", axis = 1))

In [116]:
rf_preds = []
for i in range(len(prediction)):
    rf_preds.append(prediction[i].argsort()[-5:][::-1])

In [117]:
target = [[l] for l in test_rf["hotel_cluster"]]
print(metrics.mapk(target, rf_preds, k=5))

0.181968333333


In [118]:
importances = rf_clf.feature_importances_
indices = np.argsort(importances)[::-1]
for i in indices:
    print(str(train_rf.columns[i]) + ': ' + str(importances[i]))

user_id: 0.0861349680292
Unnamed: 0.1: 0.0860743225167
user_location_city: 0.0810520040659
srch_destination_id: 0.0779443847076
hotel_market: 0.0758511219674
srch_month: 0.0757924788461
user_location_region: 0.0671519380423
orig_destination_distance: 0.0658248575423
year: 0.0586712504717
srch_yr: 0.0580162498749
month: 0.0376286542657
channel: 0.0375236671501
srch_adults_cnt: 0.0310279725281
user_location_country: 0.0243583273839
hotel_country: 0.023783047421
srch_destination_type_id: 0.0218025796399
site_name: 0.0181280855922
srch_children_cnt: 0.0171160095105
is_mobile: 0.0113421980373
srch_rm_cnt: 0.0108461963102
hotel_continent: 0.00998129664495
posa_continent: 0.0078216992503
is_package: 0.00698388416624
hotel_nights: 0.00652309856068
cnt: 0.00261970747482
is_booking: 0.0
hotel_cluster: 0.0


### Data Leak trial

In [106]:
train = pd.read_csv("500k_train.csv")
test = pd.read_csv("500k_test.csv")
match_cols = ['user_location_country', 'user_location_region', 'user_location_city', 'hotel_market', 'orig_destination_distance']

In [34]:
from itertools import chain, combinations
from collections import defaultdict
from heapq import nlargest
from operator import itemgetter
import math

subsets = []
def all_subsets(ss):
  return (chain(*map(lambda x: combinations(ss, x), range(0, len(ss)+1))))

for subset in all_subsets(match_cols):
  subsets.append(list(subset))
subsets = subsets[1:]

NameError: name 'match_cols' is not defined

In [108]:
subsets

[['user_location_country'],
 ['user_location_region'],
 ['user_location_city'],
 ['hotel_market'],
 ['orig_destination_distance'],
 ['user_location_country', 'user_location_region'],
 ['user_location_country', 'user_location_city'],
 ['user_location_country', 'hotel_market'],
 ['user_location_country', 'orig_destination_distance'],
 ['user_location_region', 'user_location_city'],
 ['user_location_region', 'hotel_market'],
 ['user_location_region', 'orig_destination_distance'],
 ['user_location_city', 'hotel_market'],
 ['user_location_city', 'orig_destination_distance'],
 ['hotel_market', 'orig_destination_distance'],
 ['user_location_country', 'user_location_region', 'user_location_city'],
 ['user_location_country', 'user_location_region', 'hotel_market'],
 ['user_location_country',
  'user_location_region',
  'orig_destination_distance'],
 ['user_location_country', 'user_location_city', 'hotel_market'],
 ['user_location_country', 'user_location_city', 'orig_destination_distance'],
 ['

In [109]:
train[subsets[29]]

,user_location_region,user_location_city,hotel_market,orig_destination_distance
0,824,46507,43,1342.0718
1,260,51299,628,1616.2906
2,174,11554,110,2369.6053
3,348,48862,489,611.6781
4,462,27117,660,148.9714
5,196,2428,1230,2436.8130
6,337,3721,675,NaN
7,657,51077,1594,NaN
8,196,2428,478,955.0108
9,135,36086,13,4685.0701


In [110]:
best_subset_name = ''
best_subset_value = 0
#best_combo = defaultdict(lambda: defaultdict(float))
for j in range(len(subsets)):
    best_combo = defaultdict(lambda: defaultdict(int))
    popular_hotel_cluster = defaultdict(int)
    for index, row in train.iterrows():
        values = []
        skip = False
        hotel_cluster = row['hotel_cluster']
        length_of_subset = len(subsets[j])
        for i in range(length_of_subset): 
            if math.isnan(row[subsets[j][i]]):
                skip = True
        if not skip:
            for i in range(length_of_subset): 
                values.append(row[subsets[j][i]])
            if length_of_subset == 1:
                best_combo[(values[0])][hotel_cluster] += 1
            elif length_of_subset == 2:
                best_combo[(values[0], values[1])][hotel_cluster] += 1
            elif length_of_subset == 3:
                best_combo[(values[0], values[1], values[2])][hotel_cluster] += 1
            elif length_of_subset == 4:
                best_combo[(values[0], values[1], values[2], values[3])][hotel_cluster] += 1
            else:
                best_combo[(values[0], values[1], values[2], values[3], values[4])][hotel_cluster] += 1
            popular_hotel_cluster[hotel_cluster] += 1
    topclusters = nlargest(5, sorted(popular_hotel_cluster.items()), key=itemgetter(1))
    preds = []
    for index, row in test.iterrows():
        values = []
        filled = []
        skip = False
        hotel_cluster = row['hotel_cluster']
        length_of_subset = len(subsets[13])
        for i in range(length_of_subset): 
            if math.isnan(row[subsets[13][i]]):
                skip = True
        if not skip:
            for i in range(length_of_subset): 
                values.append(row[subsets[13][i]])
            if length_of_subset == 1:
                s1 = values[0]
            elif length_of_subset == 2:
                s1 = (values[0], values[1])
            elif length_of_subset == 3:
                s1 = (values[0], values[1], values[2])
            elif length_of_subset == 4:
                s1 = (values[0], values[1], values[2], values[3])
            else:
                s1 = (values[0], values[1], values[2], values[3], values[4])
            output = []
            if s1 in best_combo:
                #print(s1)
                individual_outputs = []
                clusters = best_combo[s1]
                #print(clusters)
                topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
                #print(topitems)
                for i in range(len(topitems)):
                    if topitems[i][0] in filled:
                        continue
                    if len(filled) == 5:
                        break
                    output.append(str(topitems[i][0]))
                    filled.append(topitems[i][0])
                #print(output)
            for i in range(len(topclusters)):
                #print(output)
                if topclusters[i][0] in filled:
                    continue
                if len(filled) == 5:
                    break
                output.append(str(topclusters[i][0]))
                filled.append(topclusters[i][0])
            preds.append(output)
    preds_file = []
    for i in range(len(preds)):
        row_pred = []
        row_pred.append(int(preds[i][0]))
        row_pred.append(int(preds[i][1]))
        row_pred.append(int(preds[i][2]))
        row_pred.append(int(preds[i][3]))
        row_pred.append(int(preds[i][4]))
        preds_file.append(row_pred)
    target = [[l] for l in test["hotel_cluster"]]
    print(str(subsets[j]) + ':' + str(metrics.mapk(target, preds_file, k=5)))
    if metrics.mapk(target, preds_file, k=5) >= best_subset_value:
        best_subset_value = metrics.mapk(target, preds_file, k=5)
        best_subset_name = str(subsets[j])

['user_location_country']:0.0723388868731
['user_location_region']:0.0723388868731
['user_location_city']:0.0723388868731
['hotel_market']:0.0723388868731
['orig_destination_distance']:0.0718157857604
['user_location_country', 'user_location_region']:0.0723388868731
['user_location_country', 'user_location_city']:0.0723388868731
['user_location_country', 'hotel_market']:0.0723388868731
['user_location_country', 'orig_destination_distance']:0.0718157857604
['user_location_region', 'user_location_city']:0.0723388868731
['user_location_region', 'hotel_market']:0.0723388868731
['user_location_region', 'orig_destination_distance']:0.0718157857604
['user_location_city', 'hotel_market']:0.0723388868731
['user_location_city', 'orig_destination_distance']:0.0691133486534
['hotel_market', 'orig_destination_distance']:0.0718157857604
['user_location_country', 'user_location_region', 'user_location_city']:0.0723388868731
['user_location_country', 'user_location_region', 'hotel_market']:0.072338886

KeyboardInterrupt: 

In [ ]:
print(best_subset_name) 
print(best_subset_value)

In [10]:
del subsets[30]

In [11]:
subsets

[['user_location_country'],
 ['user_location_region'],
 ['user_location_city'],
 ['hotel_market'],
 ['orig_destination_distance'],
 ['user_location_country', 'user_location_region'],
 ['user_location_country', 'user_location_city'],
 ['user_location_country', 'hotel_market'],
 ['user_location_country', 'orig_destination_distance'],
 ['user_location_region', 'user_location_city'],
 ['user_location_region', 'hotel_market'],
 ['user_location_region', 'orig_destination_distance'],
 ['user_location_city', 'hotel_market'],
 ['user_location_city', 'orig_destination_distance'],
 ['hotel_market', 'orig_destination_distance'],
 ['user_location_country', 'user_location_region', 'user_location_city'],
 ['user_location_country', 'user_location_region', 'hotel_market'],
 ['user_location_country',
  'user_location_region',
  'orig_destination_distance'],
 ['user_location_country', 'user_location_city', 'hotel_market'],
 ['user_location_country', 'user_location_city', 'orig_destination_distance'],
 ['

In [13]:
best_subset_name = ''
best_subset_value = 0
#best_combo = defaultdict(lambda: defaultdict(float))
#top_dict = defaultdict(lambda: defaultdict(float))
for j in range(len(subsets)):
    top_dict = defaultdict(lambda: defaultdict(float))
    best_combo = defaultdict(lambda: defaultdict(int))
    popular_hotel_cluster = defaultdict(int)
    for index, row in train.iterrows():
        values = []
        skip = False
        hotel_cluster = row['hotel_cluster']
        length_of_subset = len(subsets[j])
        for i in range(length_of_subset): 
            if math.isnan(row[subsets[j][i]]):
                skip = True
        if not skip:
            for i in range(length_of_subset): 
                values.append(row[subsets[j][i]])
            top_dict[(row['user_location_country'], row['user_location_region'], row['user_location_city'], row['hotel_market'], row['orig_destination_distance'])][row['hotel_cluster']] += 1
            if length_of_subset == 1:
                best_combo[(values[0])][hotel_cluster] += 1
            elif length_of_subset == 2:
                best_combo[(values[0], values[1])][hotel_cluster] += 1
            elif length_of_subset == 3:
                best_combo[(values[0], values[1], values[2])][hotel_cluster] += 1
            elif length_of_subset == 4:
                best_combo[(values[0], values[1], values[2], values[3])][hotel_cluster] += 1
            else:
                best_combo[(values[0], values[1], values[2], values[3], values[4])][hotel_cluster] += 1
            popular_hotel_cluster[hotel_cluster] += 1
    topclusters = nlargest(5, sorted(popular_hotel_cluster.items()), key=itemgetter(1))
    preds = []
    for index, row in test.iterrows():
        values = []
        filled = []
        skip = False
        hotel_cluster = row['hotel_cluster']
        length_of_subset = len(subsets[13])
        for i in range(length_of_subset): 
            if math.isnan(row[subsets[13][i]]):
                skip = True
        if not skip:
            for i in range(length_of_subset): 
                values.append(row[subsets[13][i]])
            if length_of_subset == 1:
                s1 = values[0]
            elif length_of_subset == 2:
                s1 = (values[0], values[1])
            elif length_of_subset == 3:
                s1 = (values[0], values[1], values[2])
            elif length_of_subset == 4:
                s1 = (values[0], values[1], values[2], values[3])
            else:
                s1 = (values[0], values[1], values[2], values[3], values[4])
            output = []
            if (row['user_location_country'], row['user_location_region'], row['user_location_city'], row['hotel_market'], row['orig_destination_distance']) in top_dict:
                #print(s1)
                individual_outputs = []
                clusters = top_dict[row['user_location_country'], row['user_location_region'], row['user_location_city'], row['hotel_market'], row['orig_destination_distance']]
                #print(clusters)
                topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
                #print(topitems)
                for i in range(len(topitems)):
                    if topitems[i][0] in filled:
                        continue
                    if len(filled) == 5:
                        break
                    output.append(str(topitems[i][0]))
                    filled.append(topitems[i][0])
                #print(output)
            if s1 in best_combo:
                #print(s1)
                individual_outputs = []
                clusters = best_combo[s1]
                #print(clusters)
                topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
                #print(topitems)
                for i in range(len(topitems)):
                    if topitems[i][0] in filled:
                        continue
                    if len(filled) == 5:
                        break
                    output.append(str(topitems[i][0]))
                    filled.append(topitems[i][0])
                #print(output)
            for i in range(len(topclusters)):
                #print(output)
                if topclusters[i][0] in filled:
                    continue
                if len(filled) == 5:
                    break
                output.append(str(topclusters[i][0]))
                filled.append(topclusters[i][0])
            preds.append(output)
    preds_file = []
    for i in range(len(preds)):
        row_pred = []
        row_pred.append(int(preds[i][0]))
        row_pred.append(int(preds[i][1]))
        row_pred.append(int(preds[i][2]))
        row_pred.append(int(preds[i][3]))
        row_pred.append(int(preds[i][4]))
        preds_file.append(row_pred)
    target = [[l] for l in test["hotel_cluster"]]
    print(str(subsets[j]) + ':' + str(metrics.mapk(target, preds_file, k=5)))
    if metrics.mapk(target, preds_file, k=5) >= best_subset_value:
        best_subset_value = metrics.mapk(target, preds_file, k=5)
        best_subset_name = str(subsets[j])

['user_location_country']:0.0754078201074
['user_location_region']:0.0754078201074
['user_location_city']:0.0754078201074
['hotel_market']:0.0754078201074
['orig_destination_distance']:0.0753174722682
['user_location_country', 'user_location_region']:0.0754078201074
['user_location_country', 'user_location_city']:0.0754078201074
['user_location_country', 'hotel_market']:0.0754078201074
['user_location_country', 'orig_destination_distance']:0.0753174722682
['user_location_region', 'user_location_city']:0.0754078201074
['user_location_region', 'hotel_market']:0.0754078201074
['user_location_region', 'orig_destination_distance']:0.0753174722682
['user_location_city', 'hotel_market']:0.0754078201074
['user_location_city', 'orig_destination_distance']:0.0753174722682
['hotel_market', 'orig_destination_distance']:0.0753174722682
['user_location_country', 'user_location_region', 'user_location_city']:0.0754078201074
['user_location_country', 'user_location_region', 'hotel_market']:0.075407820

In [47]:
train = pd.read_csv("50k_train.csv")
test = pd.read_csv("50k_test.csv")
topclusters = list(train.hotel_cluster.value_counts().head().index)

In [48]:
from collections import defaultdict

ulc_odd = defaultdict(lambda: defaultdict(int))
all_columns = defaultdict(lambda: defaultdict(int))
sdi_hc_hm_by = defaultdict(lambda: defaultdict(int))
sdi_dict = defaultdict(lambda: defaultdict(int))
hc_dict = defaultdict(lambda: defaultdict(int))
popular_hotel_cluster = defaultdict(int)
all_minus_odd = defaultdict(lambda: defaultdict(int))
all_minus_hm = defaultdict(lambda: defaultdict(int))
all_minus_ulci = defaultdict(lambda: defaultdict(int))
all_minus_ulco = defaultdict(lambda: defaultdict(int))
all_minus_ulr = defaultdict(lambda: defaultdict(int))
for index, row in train.iterrows():
    values = []
    hotel_cluster = row['hotel_cluster']
    ulci = row['user_location_city']
    ulco = row['user_location_country']
    ulr = row['user_location_region']
    odd = row['orig_destination_distance']
    sdi = row['srch_destination_id']
    hm = row['hotel_market']
    hc = row['hotel_country']
    by = row['year']

    if ulci != '' and odd != '':
        ulc_odd[(ulci, odd)][hotel_cluster] += 1
        
    if ulci != '' and ulco != '' and ulr != '' and odd != '' and hm != '':
        all_columns[(ulci, ulco, ulr, odd, hm)][hotel_cluster] += 1
        
    if sdi != '' and hc != '' and hm != '' and by != '':
        sdi_hc_hm_by[(sdi, hc, hm, by)][hotel_cluster] += 1
        
    if ulci != '' and ulco != '' and ulr != '' and sdi != '' and hm != '':
        all_minus_odd[(ulci, ulco, ulr, sdi, hm)][hotel_cluster] += 1
        
    if ulci != '' and ulco != '' and ulr != '' and sdi != '' and odd != '':
        all_minus_hm[(ulci, ulco, ulr, sdi, odd)][hotel_cluster] += 1
    
    if hm != '' and ulco != '' and ulr != '' and sdi != '' and hm != '':
        all_minus_ulci[(hm, ulco, ulr, sdi, odd)][hotel_cluster] += 1
        
    if ulci != '' and ulco != '' and ulr != '' and sdi != '' and hm != '':
        all_minus_ulco[(hm, ulci, ulr, sdi, odd)][hotel_cluster] += 1
        
    if ulci != '' and ulco != '' and ulr != '' and sdi != '' and hm != '':
        all_minus_ulr[(hm, ulci, ulco, sdi, odd)][hotel_cluster] += 1
        
    if sdi != '':
        sdi_dict[sdi][hotel_cluster] += 1

    if hc != '':
        hc_dict[hc][hotel_cluster] += 1

In [49]:
preds = []
indicies = []
for index, row in test.iterrows():
    output = []
    filled = []
    run_ml = True
    hotel_cluster = row['hotel_cluster']
    ulci = row['user_location_city']
    ulco = row['user_location_country']
    ulr = row['user_location_region']
    odd = row['orig_destination_distance']
    sdi = row['srch_destination_id']
    hm = row['hotel_market']
    hc = row['hotel_country']
    by = row['year']
    
    if (ulci, odd) in ulc_odd:
        run_ml = False
        individual_outputs = []
        clusters = ulc_odd[ulci, odd]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
    '''       
    if (ulci, ulco, ulr, odd, hm) in all_columns:
        run_ml = False
        individual_outputs = []
        clusters = all_columns[ulci, ulco, ulr, odd, hm]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
    
    if (ulci, ulco, ulr, sdi, hm) in all_columns:
        run_ml = False
        individual_outputs = []
        clusters = all_minus_odd[ulci, ulco, ulr, sdi, hm]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
            
    if (ulci, ulco, ulr, sdi, odd) in all_columns:
        run_ml = False
        individual_outputs = []
        clusters = all_minus_hm[ulci, ulco, ulr, sdi, odd]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
            
    if (hm, ulco, ulr, sdi, odd) in all_columns:
        run_ml = False
        individual_outputs = []
        clusters = all_minus_ulci[hm, ulco, ulr, sdi, odd]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
            
    if (hm, ulci, ulr, sdi, odd) in all_columns:
        run_ml = False
        individual_outputs = []
        clusters = all_minus_ulco[hm, ulci, ulr, sdi, odd]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
            
    if (hm, ulci, ulco, sdi, odd) in all_columns:
        run_ml = False
        individual_outputs = []
        clusters = all_minus_ulr[hm, ulci, ulco, sdi, odd]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
    '''

    if (sdi, hc, hm, by) in sdi_hc_hm_by:
        run_ml = False
        individual_outputs = []
        clusters = sdi_hc_hm_by[sdi, hc, hm, by]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
            
    '''
    if sdi in sdi_dict:
        run_ml = False
        individual_outputs = []
        clusters = sdi_dict[sdi]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
            
    if hc in hc_dict:
        run_ml = False
        individual_outputs = []
        clusters = hc_dict[hc]
        topitems = nlargest(5, sorted(clusters.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            output.append(str(topitems[i][0]))
            filled.append(topitems[i][0])
    '''
    
    for i in range(len(topclusters)):
        if topclusters[i] in filled:
            continue
        if len(filled) == 5:
            break
        output.append(str(topclusters[i]))
        filled.append(topclusters[i])
    if run_ml:
        indicies.append(index)
    preds.append(output)

In [50]:
preds_file = []
for i in range(len(preds)):
    row_pred = []
    row_pred.append(int(preds[i][0]))
    row_pred.append(int(preds[i][1]))
    row_pred.append(int(preds[i][2]))
    row_pred.append(int(preds[i][3]))
    row_pred.append(int(preds[i][4]))
    preds_file.append(row_pred)

In [51]:
target = [[l] for l in test["hotel_cluster"]]
print(metrics.mapk(target, preds_file, k=5))

0.249236666667


In [58]:
for i in indicies:
    preds_file[i][0] = rf_preds[i][0]
    preds_file[i][1] = rf_preds[i][1]
    preds_file[i][2] = rf_preds[i][2]
    preds_file[i][3] = rf_preds[i][3]
    preds_file[i][4] = rf_preds[i][4]

In [59]:
target = [[l] for l in test["hotel_cluster"]]
print(metrics.mapk(target, preds_file, k=5))

0.254181666667


In [77]:
for i in indicies:
    preds_file[i][0] = xgb_preds[i][0]
    preds_file[i][1] = xgb_preds[i][1]
    preds_file[i][2] = xgb_preds[i][2]
    preds_file[i][3] = xgb_preds[i][3]
    preds_file[i][4] = xgb_preds[i][4]
target = [[l] for l in test["hotel_cluster"]]
print(metrics.mapk(target, preds_file, k=5))

0.257725


In [103]:
pred_file = pd.read_csv('submission_2017-12-01-20-47.csv')
large_test = pd.read_csv('50k_test.csv')

In [104]:
preds_file = []
for i in range(len(pred_file['hotel_cluster'])):
    row_pred = []
    arr = pred_file['hotel_cluster'][i].split(" ")
    row_pred.append(int(arr[1]))
    row_pred.append(int(arr[2]))
    row_pred.append(int(arr[3]))
    row_pred.append(int(arr[4]))
    row_pred.append(int(arr[5]))
    preds_file.append(row_pred)

In [45]:
target = [[l] for l in large_test["hotel_cluster"]]
print(metrics.mapk(target, preds_file, k=5))

NameError: name 'large_test' is not defined